In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [3]:
graphPath = "/data/graphDFSample"

In [4]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number,sort_array,udf
from pyspark.sql import Window
from pyspark.sql.types import LongType,ArrayType

def list_pairs(users):
    pairs=[]
    for user in users:
        for pair in users:
            if user != pair and user<pair:
                pairs.append((user,pair))
    return pairs

list_pairs_udf = udf(list_pairs,ArrayType(ArrayType(LongType())))

In [5]:
reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users")) \
    .withColumn("users_size", size("users")) \
    .withColumn("sorted_user", sort_array("users"))\
    .where("users_size > 1")\
    .withColumn("pairs",list_pairs_udf("sorted_user"))\
    .withColumn("all_pairs",explode("pairs"))

In [6]:
window = Window.orderBy(col("count").desc())

In [16]:
top50 = reversedGraph.groupBy("all_pairs")\
            .count()\
            .withColumn("row_number", row_number().over(window)) \
            .filter(col("row_number") < 50) \
            .orderBy(col("count").desc()) \
            .select("all_pairs","count") \
            .collect()

In [23]:
for pair,count in top50:
    print '%s %s %s' % (count , pair[0],pair[1])

901 13878153 64040607
482 39037059 62716987
200 9825755 13730644
60 13730644 13802436
49 9825755 13802436
45 9825755 24320891
28 5582333 45023113
26 5796650 13730644
26 5796650 13802436
20 13730644 24320891
20 6021591 61751428
19 24320891 37450332
17 5796650 9825755
17 6021591 34970445
15 37408934 37821698
14 22312953 37735419
14 19071788 22991438
13 9825755 37408934
11 13730644 37408934
10 29439114 61751428
9 13802436 24320891
9 9825755 20871861
8 20871861 24320891
8 13878153 62716987
8 62716987 64040607
8 9825755 16958135
7 37735419 61751428
7 18094770 31863190
6 13878153 39037059
6 39037059 64040607
6 9825755 18094770
6 12886695 31863190
6 9825755 22991438
6 9825755 37450332
6 13730644 37735419
6 13730644 61751428
6 13730644 35448979
6 22312953 61751428
5 13730644 20871861
5 13802436 22991438
5 9825755 61751428
5 9825755 38137713
5 15002145 38137713
5 13802436 20871861
5 34970445 61751428
5 22991438 61751428
5 12886695 18094770
4 28404521 35436637
4 13730644 18094770
